**Что ищем**: 1к квартиру или студию до 14к\мес с мебелью и минимальной комиссией риелтора

**Где ищем**: 
  - Дзержинский (Парковый, Светлый, Цетр, Данилиха)
  - Индустриальный (Балатово, Новоплоский)
  - Свердловский (Островский, Громовский, Свердловский)
  - Ленинский (Центр)
  - Мотовилихинский (Рабочий посёлок, Городские горки)

In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import re
import sys

In [2]:
# в url зашиты микрорайоны и однокомнатность 
listurl = "http://perm.n1.ru/search/?rubric=flats&deal_type=rent_out&include_suburbs=false&microdistrict=\
4846771%2C4846776%2C4846777%2C4846779%2C4846780%2C4846785%2C4846802%2C4846807%2C4846840%2C4846841%2C4846842&\
rooms=1&creation_date_min=3&sort=price"

In [3]:
#отправляем запрос по url и возвращаем html-страницу как текст
def get_html_text(html_url, save_to_disc = False):
    r = requests.get(html_url)
    text = r.text.encode('utf-8')
    if save_to_disc == True:
        with open('test.html', 'w') as output_file:
          output_file.write(text)
    return text
# заменяем тэги <br> на перенос строки в текстовом html-элементе, возвращаем нормальный текст
def replace_br(elem, br_replacement = '\n'):
    text = ''
    if not (elem is None):
        for e in elem.recursiveChildGenerator():
            if isinstance(e, str):
                text += e.strip()
            elif e.name == 'br':
                text += br_replacement
    return text
# на входе html-страница со списком квартир
# на выходе список ссылок на каждую квартиру
def get_all_flats_urls(list_html_text):
    soup = BeautifulSoup(list_html_text, "lxml")
    addr_list = soup.find('div', {'class': 'offers-search'})
    items = addr_list.find_all('div', {'class': 'living-list-card__title living-list-card-title'})
    return map(lambda addr_div: "http://perm.n1.ru"+addr_div.find('a', {'class': 'living-list-card-title__link'})\
               .get('href'), items)

# на входе ссылка на список квартир
# на выходе список с параметрами каждой квартиры
def get_all_flats_data(list_url, max_urls=100):
    results = []
    list_html_text = get_html_text(list_url)
    flats_urls = get_all_flats_urls(list_html_text)
    for i, flat_url in enumerate(flats_urls):
        if i >= max_urls:
            break
        results.append(get_flat_data(flat_url))
    return results

def elem_text(elem):
    if not (elem is None):
        return elem.text#.encode('utf-8')
    else:
        return "Н\Д"
def elem_numeric(elem):
    non_decimal = re.compile(r'[^\d.]+')
    elem_txt = non_decimal.sub('', elem_text(elem))
    try:
        return int(elem_txt)
    except:
        return 0

# на входе ссылка на квартиру
# на выходе параметры квартиры в виде одной строки
def get_flat_data(flat_url):
    flat_html_text = get_html_text(flat_url)
    soup = BeautifulSoup(flat_html_text, "lxml")
    
    addr = elem_text(soup.find('h1', {'class': 'card-living-content-header__title'}))
    addr = addr.replace('Сдам ','').replace("1-к,","")

    price = elem_numeric(soup.find('span', {'class': 'price__val'})) # 'card-living-content-price__val'}))

    owner = elem_text(soup.find('div', {'class': 'offer-card-contacts__person _type'}))
    owner+= " "
    owner+= elem_text(soup.find('div', {'class': 'offer-card-contacts__person'}))

    descr_elem = soup.find('div', {'class': 'offer-card-description__text'})
    descr = replace_br(descr_elem)

    params_dict = {}
    params_list = soup.findAll('li', {'class': 'card-living-content-params-list__item'})
    for param in params_list:
        param_name = elem_text(param.find('span', {'class': 'card-living-content-params-list__name'}))
        param_val = elem_text(param.find('span', {'class': 'card-living-content-params-list__value'}))
        params_dict[param_name] = param_val
    
    area = params_dict.get('Общая площадь', 'Н\Д')
    floor = params_dict.get('Этаж', 'Н\Д')
    fridge = params_dict.get('Холодильник', 'Н\Д')
    furniture = params_dict.get('Мебель', 'Н\Д')
    return {
                'url': flat_url,
                'addr': addr,
                'price': price,
                'owner': owner,
                'area': area,
                'floor': floor,
                'fridge': fridge,
                'furniture': furniture,
                'descr': descr
            }

In [4]:
list_html_text = get_html_text(listurl)
print(list(get_all_flats_urls(list_html_text)))

['http://perm.n1.ru/view/28603272/', 'http://perm.n1.ru/view/28584156/', 'http://perm.n1.ru/view/28590711/', 'http://perm.n1.ru/view/28603674/', 'http://perm.n1.ru/view/28589282/', 'http://perm.n1.ru/view/28602359/', 'http://perm.n1.ru/view/28616467/', 'http://perm.n1.ru/view/28615949/', 'http://perm.n1.ru/view/28605705/', 'http://perm.n1.ru/view/28605372/', 'http://perm.n1.ru/view/28584825/', 'http://perm.n1.ru/view/28584837/', 'http://perm.n1.ru/view/28616490/', 'http://perm.n1.ru/view/28605021/', 'http://perm.n1.ru/view/28591966/', 'http://perm.n1.ru/view/28589191/', 'http://perm.n1.ru/view/28616044/', 'http://perm.n1.ru/view/28605312/', 'http://perm.n1.ru/view/28624785/', 'http://perm.n1.ru/view/28618203/', 'http://perm.n1.ru/view/28584166/', 'http://perm.n1.ru/view/28617024/', 'http://perm.n1.ru/view/28617513/', 'http://perm.n1.ru/view/28615254/', 'http://perm.n1.ru/view/28623538/']


In [7]:
flats = get_all_flats_data(listurl, max_urls=50)

In [8]:
flats_df = pd.DataFrame(flats)

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

flats_df.style.format({'url': make_clickable}) 

,addr,area,descr,floor,fridge,furniture,owner,price,url
0,"Клары Цеткин, 27",25 м2,"Уютная 1 комнатная мало семейка студия , хороший ремонт, делался для себя. Приятная атмосфера. Есть вся необходимая мебель. Своя кухня, светлый кухонный гарнитур, холодильник, микроволновка. Санузел свой, стиральная машинка. Сдается аккуратным, порядочным людям",4 из 9,есть,нет,Собственник Собственник,7000,http://perm.n1.ru/view/28603272/
1,"Гагарина бульвар, 32а",17 м2,"Квартира -студия в хорошем состоянии, удобное транспортное расположение",1 из 9,есть,есть,Собственник Собственник,8000,http://perm.n1.ru/view/28584156/
2,"Пушкина, 109",42 м2,"Сдам однокомнатную квартиру .Квартира меблирована , на кухне: кухонный гарнитур ,обеденный уголок ,холодильник .В комнате :шкаф купе ,диван ,два кресла ,телевизор ,в ванной машинка автомат и мелкая бытовая техника . В стоимость входят и коммунальные услуги ,цена приемлема ,звоните!",6 из 18,есть,есть,Собственник Собственник,8500,http://perm.n1.ru/view/28590711/
3,"Карпинского, 38",31 м2,"Срочно сдам 1 ком квартиру на длительный срок. Тёплая, чистая, мебель, техника, балкон не застеклен, железная дверь. Желающих рассмотрим. На длительный срок",4 из 5,есть,есть,Собственник Собственник,8500,http://perm.n1.ru/view/28603674/
4,"Мира, 8б",38 м2,"Сдаётся переделанная 1 комнатная квартира в студию, несмотря на то, что соединили балкон и квартиру, очень тепло, нет сквозняков, вся мебель остаётся, посуда, постельное так же имеется. В первую очередь рассматриваем семейные пары, либо одну девушку или одного молодого человека, толпой сразу нет.",5 из 9,есть,есть,Собственник Собственник,8500,http://perm.n1.ru/view/28589282/
5,"Красноармейская 1-я, 5",44 м2,"Сдам квартиру на долгосрочный период. Коммунальные платежи включены в стоимость. Не важно, кто будет жить, главное своевременная плата. Мебель и техника вся имеется. Звоните и приходите.",12 из 17,есть,есть,Собственник Собственник,8500,http://perm.n1.ru/view/28602359/
6,"Семченко, 6",45 м2,"Сдам однокомнатную квартиру в отличном состоянии рядом с ТЦ XXI Век, СПИ внешнеэкономических связей, экономики и права. В квартире есть вся мебель и бытовая техника для комфортного проживания. Отличная транспортная развязка, так же рядом находится детский садик, поликлиника. Есть кабельное ТВ и интернет (оплачивается отдельно). Рассмотрю любую кандидатуру. Квартира готова к заселению.",7 из 14,есть,есть,Частный риелтор Частный риелтор,9000,http://perm.n1.ru/view/28616467/
7,"Краснофлотская, 28",40 м2,"Срочно сдам солнечную однушку в спокойном квартале, в 5 минутах ходьбы от остановки общественного транспорта. В квартире есть вся мебель и техника, необходимая для комфортной жизни. Диван, шкаф, стол, цифровое ТВ, ремонт делали для себя, есть балкон. На кухне есть плита, холодильник, чайник, стиральная машина. Соседи тихие, в шаговой доступности продуктовый и общественный транспорт, отделение банка, ТК. Сдаю только на длительный срок, платежеспособным людям. Желательно паре, но рассмотрю все варианты. Без залога. Звонить в дневное время.",9 из 16,есть,есть,Собственник Собственник,10000,http://perm.n1.ru/view/28615949/
8,"Мира, 55",30 м2,"Сдам однокомнатную квартиру от собственника. Квартира мебелирована, имеется диван, холодильник, стиральная машина. Оплата 10.000 руб. в месяц плюс коммунальные услуги.",3 из 5,есть,есть,Собственник Собственник,10000,http://perm.n1.ru/view/28605705/
9,"Парковый проспект, 25г",39 м2,"Отличная квартира в центре паркового. До центра города 10 минут, рядом множество садиков, школ, магазинов. Сдаём на длительный срок. Подходим основательно. Все вопросы строго по телефону.",7 из 9,есть,есть,Собственник Собственник,10000,http://perm.n1.ru/view/28605372/
